# 03 - Golden Layer: Feature Engineering & Embeddings

In [0]:
import pandas as pd
import numpy as np
from pyspark.sql import functions
from pyspark.sql.types import StructType, StructField, IntegerType, LongType, DoubleType, StringType, ArrayType, FloatType

In [0]:
try:
    from sentence_transformers import SentenceTransformer
except ImportError:
    import sys
    !{sys.executable} -m pip install -q sentence-transformers
    from sentence_transformers import SentenceTransformer

Configurations

In [0]:
SILVER_TABLE = "anime_project.silver.anime_enriched"
GOLD_FEATURES_TABLE = "anime_project.golden.anime_features"
GOLD_EMBEDDINGS_TABLE = "anime_project.golden.anime_embeddings"

print("📂 Configuration:")
print(f"   Silver input: {SILVER_TABLE}")
print(f"   Gold features output: {GOLD_FEATURES_TABLE}")
print(f"   Gold embeddings output: {GOLD_EMBEDDINGS_TABLE}")

Load Silver Layer

In [0]:
df_silver = spark.table(SILVER_TABLE)

print(f"📊 Silver Table Loaded: {SILVER_TABLE}")
print(f"   Total rows: {df_silver.count()}")
print(f"   Total columns: {len(df_silver.columns)}")

Convert to Pandas for easier feature engineering

In [0]:
df_pandas = df_silver.toPandas()
print(f"\n✅ Converted to Pandas: {df_pandas.shape}")

In [0]:
print(f"\nAvailable Columns:")
for i, col in enumerate(df_pandas.columns, 1):
    print(f"   {i:2d}. {col}")

Parse Comma-Separated Fields (Genres, Themes, Studios)

In [0]:
df_parsed = df_silver \
    .withColumn("genres_array", functions.split(functions.col("genres"), ", ")) \
    .withColumn("themes_array", functions.split(functions.col("themes"), ", ")) \
    .withColumn("studios_array", functions.split(functions.col("studios"), ", ")).filter(functions.col("status") == "Completed")

print("✅ Parsed comma-separated fields into arrays")

# Show sample to verify
print("\nSample with parsed arrays:")
df_parsed.select(
    "anime_id",
    "anime_title",  
    "genres_array",
    "studios_array",
    "themes_array"
).show(5, truncate=False)

In [0]:
print("\n📊 Array Statistics:")
df_parsed.select(
    functions.avg(functions.size("genres_array")).alias("avg_genres_per_anime"),
    functions.max(functions.size("genres_array")).alias("max_genres"),
    functions.avg(functions.size("themes_array")).alias("avg_themes_per_anime"),
    functions.avg(functions.size("studios_array")).alias("avg_studios_per_anime")
).show()

Calculate Genre & Studio Preferences

In [0]:
df_genres_exploded = df_parsed \
    .select("anime_id", "user_id", "user_score", functions.explode("genres_array").alias("genre")) \
    .filter(functions.col("user_score").isNotNull())  # Only scored anime

print("✅ Exploded genres array")
print(f"   Original rows: {df_parsed.count()}")
print(f"   Exploded rows: {df_genres_exploded.count()} (one per genre)")


In [0]:
print("\n👀 Sample exploded data:")
df_genres_exploded.show(10, truncate=False)

In [0]:
genre_preferences = df_genres_exploded \
    .groupBy("genre") \
    .agg(
        functions.avg("user_score").alias("avg_genre_score"),
        functions.count("*").alias("genre_count")
    ) \
    .orderBy(functions.desc("avg_genre_score"))

print("\n⭐ Your Genre Preferences (by average score):")
genre_preferences.show(20, truncate=False)


In [0]:
df_studios_exploded = df_parsed \
    .select("anime_id", "user_id", "user_score", functions.explode("studios_array").alias("studio")) \
    .filter(functions.col("user_score").isNotNull())

studio_preferences = df_studios_exploded \
    .groupBy("studio") \
    .agg(
        functions.avg("user_score").alias("avg_studio_score"),
        functions.count("*").alias("studio_count")
    ) \
    .filter(functions.col("studio_count") >= 2).orderBy(functions.desc("avg_studio_score"))

In [0]:
print("\n🎬 Studio Preferences (2+ anime only):")
studio_preferences.show(20, truncate=False)

In [0]:
genre_preferences.printSchema()
print(f"Genre prefs rows: {genre_preferences.count()}")

In [0]:
studio_preferences.printSchema()
print(f"Studio prefs rows: {studio_preferences.count()}")

### Join Preferences Back to Main Data & Create Feature Columns

Explode genres and join with genre preferences

In [0]:
df_genres_with_prefs = df_genres_exploded.alias("exploded") \
    .join(
        genre_preferences.alias("prefs"),
        functions.col("exploded.genre") == functions.col("prefs.genre"),
        "left"
    ) \
    .select("exploded.anime_id", "exploded.user_score", "exploded.genre", "prefs.avg_genre_score")

In [0]:
genre_features = df_genres_with_prefs \
    .groupBy("anime_id") \
    .agg(
        functions.avg("avg_genre_score").alias("avg_genre_preference_score"),
        functions.count("genre").alias("num_genres_matched")
    )

print("Genre features created:")
genre_features.show(5)
print(f"Total anime with genre features: {genre_features.count()}")

In [0]:
df_studios_with_prefs = df_studios_exploded.alias("studio_exploded") \
    .join(
        studio_preferences.alias("studio_prefs"),
        functions.col("studio_exploded.studio") == functions.col("studio_prefs.studio"),
        "left"
    ) \
    .select("studio_exploded.anime_id", "studio_exploded.user_score", "studio_exploded.studio", "studio_prefs.avg_studio_score")

In [0]:
studio_features = df_studios_with_prefs \
    .groupBy("anime_id") \
    .agg(
        functions.avg("avg_studio_score").alias("avg_studio_preference_score"),
        functions.count("studio").alias("num_studios_matched")
    )

print("\nStudio features created:")
studio_features.show(5)
print(f"Total anime with studio features: {studio_features.count()}")

In [0]:
df_with_all_features = df_parsed \
    .join(genre_features, on="anime_id", how="left") \
    .join(studio_features, on="anime_id", how="left")

print("\nFinal dataframe with all features:")
df_with_all_features.printSchema()
print(f"Total rows: {df_with_all_features.count()}")

In [0]:
print("\nNull counts in new feature columns:")
df_with_all_features.select([
    functions.count(functions.when(functions.col(c).isNull(), c)).alias(c) 
    for c in ["avg_genre_preference_score", "avg_studio_preference_score", "num_genres_matched", "num_studios_matched"]
]).show()

In [0]:
print("\nSample rows with new features:")
df_with_all_features.select(
    "anime_id", 
    "anime_title", 
    "user_score",
    "avg_genre_preference_score",
    "avg_studio_preference_score",
    "num_genres_matched",
    "num_studios_matched"
).show(5)

### Add Metadata & Text-based Features

In [0]:
df_with_metadata_features = df_with_all_features \
    .withColumn("synopsis_length", functions.length(functions.col("synopsis"))) \
    .withColumn("has_synopsis", functions.when(functions.col("synopsis").isNotNull(), 1).otherwise(0)) \
    .withColumn("num_genres", functions.size(functions.col("genres_array"))) \
    .withColumn("num_themes", functions.when(functions.col("themes").isNotNull(), functions.size(functions.split(functions.col("themes"), ", "))).otherwise(0)) \
    .withColumn("num_studios", functions.size(functions.col("studios_array"))) \
    .withColumn("score_normalized", functions.col("score") / 10.0)

In [0]:
print("Step 1: Metadata features added")
df_with_metadata_features.select(
    "anime_id",
    "user_score",
    "synopsis_length",
    "has_synopsis",
    "num_genres",
    "num_themes",
    "num_studios",
    "score_normalized"
).show(5)

In [0]:
df_filled_nulls = df_with_metadata_features \
    .fillna(0.0, subset=["avg_genre_preference_score", "avg_studio_preference_score"]) \
    .fillna(0, subset=["num_genres_matched", "num_studios_matched"])

print("\nStep 2: Nulls filled with 0")
print(f"Total rows: {df_filled_nulls.count()}")

In [0]:
df_with_composite_features = df_filled_nulls \
    .withColumn(
        "user_preference_alignment",
        (functions.col("avg_genre_preference_score") + functions.col("avg_studio_preference_score")) / 2
    ) \
    .withColumn(
        "external_quality_score",
        functions.col("score_normalized")  # anime's MAL score (normalized)
    ) \
    .withColumn(
        "content_richness",
        (functions.col("num_genres") + functions.col("num_themes") + functions.col("num_studios")) / 3.0
    )

print("\nStep 3: Composite features created")
df_with_composite_features.select(
    "anime_id",
    "anime_title",
    "user_score",
    "user_preference_alignment",
    "external_quality_score",
    "content_richness"
).show(5)

In [0]:
df_with_popularity = df_with_composite_features \
    .withColumn(
        "popularity_bucket",
        functions.when(functions.col("popularity") <= 100, "very_popular") \
        .when(functions.col("popularity") <= 500, "popular") \
        .when(functions.col("popularity") <= 2000, "moderate") \
        .otherwise("niche")
    )

print("\nStep 4: Popularity buckets added")
df_with_popularity.groupBy("popularity_bucket").count().show()

In [0]:
df_final_features = df_with_popularity.select(
    "user_id",
    "anime_id",
    "anime_title",
    "user_score",
    "synopsis",
    "genres",
    "themes",
    "studios",
    "score",
    "rating",
    "source",
    "season",
    "year",
    "synopsis_length",
    "has_synopsis",
    "num_genres",
    "num_themes",
    "num_studios",
    "avg_genre_preference_score",
    "avg_studio_preference_score",
    "num_genres_matched",
    "num_studios_matched",
    "user_preference_alignment",
    "external_quality_score",
    "content_richness",
    "popularity_bucket"
)

print("\nStep 5: Final feature set ready for embeddings")
df_final_features.printSchema()
print(f"Total rows: {df_final_features.count()}")
print(f"Total columns: {len(df_final_features.columns)}")

In [0]:
print("\nStep 6: Feature Summary Statistics")
df_final_features.select(
    functions.avg("user_score").alias("avg_user_score"),
    functions.avg("user_preference_alignment").alias("avg_preference_alignment"),
    functions.avg("external_quality_score").alias("avg_external_quality"),
    functions.avg("content_richness").alias("avg_content_richness"),
    functions.min("synopsis_length").alias("min_synopsis_length"),
    functions.max("synopsis_length").alias("max_synopsis_length")
).show()

## Creating Embeddings

In [0]:
df_pandas  = df_final_features.toPandas()
print(f"Total rows: {len(df_pandas)}")
print(f"Total columns: {len(df_pandas.columns)}")

In [0]:

missing_synopsis_count = df_pandas["synopsis"].isna().sum()
print(f"Missing synopses: {missing_synopsis_count}")
df_pandas["synopsis"] = df_pandas["synopsis"].fillna("No synopsis available")

In [0]:
model = SentenceTransformer('all-MiniLM-L6-v2')
print(f"Model loaded: {model}")

In [0]:
embeddings_list = model.encode(
    df_pandas["synopsis"].tolist(),
    show_progress_bar=True,
    batch_size=32
)

print(f"Embeddings shape: {embeddings_list.shape}")
print(f"Each embedding dimension: {embeddings_list.shape[1]}")

In [0]:
df_pandas["synopsis_embedding"] = [list(emb) for emb in embeddings_list]

print(f"Total dataframe shape: {df_pandas.shape}")

In [0]:
for idx in range(min(3, len(df_pandas))):
    title = df_pandas.iloc[idx]["anime_title"]
    embedding_sample = df_pandas.iloc[idx]["synopsis_embedding"][:5]
    print(f"{title}: {embedding_sample}")

In [0]:
embedding_magnitudes = [np.linalg.norm(emb) for emb in embeddings_list]
print(f"  Embedding magnitude - Min: {min(embedding_magnitudes):.4f}, Max: {max(embedding_magnitudes):.4f}, Avg: {np.mean(embedding_magnitudes):.4f}")

In [0]:
df_with_embeddings_spark = spark.createDataFrame(
    df_pandas,
    schema=StructType([
        StructField("user_id", IntegerType(), False),  # primary key
        StructField("anime_id", LongType(), False),    # primary key
        StructField("anime_title", StringType(), True),
        StructField("user_score", LongType(), True),
        StructField("synopsis", StringType(), True),
        StructField("genres", StringType(), True),
        StructField("themes", StringType(), True),
        StructField("studios", StringType(), True),
        StructField("score", DoubleType(), True),
        StructField("rating", StringType(), True),
        StructField("source", StringType(), True),
        StructField("season", StringType(), True),
        StructField("year", DoubleType(), True),
        StructField("synopsis_length", IntegerType(), True),
        StructField("has_synopsis", IntegerType(), False),
        StructField("num_genres", IntegerType(), True),
        StructField("num_themes", IntegerType(), True),
        StructField("num_studios", IntegerType(), True),
        StructField("avg_genre_preference_score", DoubleType(), False),
        StructField("avg_studio_preference_score", DoubleType(), False),
        StructField("num_genres_matched", LongType(), False),
        StructField("num_studios_matched", LongType(), False),
        StructField("user_preference_alignment", DoubleType(), True),
        StructField("external_quality_score", DoubleType(), True),
        StructField("content_richness", DoubleType(), True),
        StructField("popularity_bucket", StringType(), False),
        StructField("synopsis_embedding", ArrayType(DoubleType()), True)
    ])
)

print(f"Spark dataframe with embeddings created")
print(f"Shape: {df_with_embeddings_spark.count()} rows")

In [0]:
df_with_embeddings_spark.printSchema()

In [0]:
df_with_embeddings_spark.select(
    "anime_id",
    "anime_title",
    "user_score",
    "user_preference_alignment",
    "synopsis_embedding"
).show(3, truncate=50)

### Save Features & Embeddings to Gold Layer

In [0]:
GOLD_FEATURES_TABLE = "anime_project.gold.anime_features"

df_with_embeddings_spark \
    .select(
        "user_id",
        "anime_id",
        "anime_title",
        "user_score",
        "synopsis",
        "genres",
        "themes",
        "studios",
        "score",
        "rating",
        "source",
        "season",
        "year",
        "synopsis_length",
        "has_synopsis",
        "num_genres",
        "num_themes",
        "num_studios",
        "avg_genre_preference_score",
        "avg_studio_preference_score",
        "num_genres_matched",
        "num_studios_matched",
        "user_preference_alignment",
        "external_quality_score",
        "content_richness",
        "popularity_bucket"
    ) \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(GOLD_FEATURES_TABLE)

print(f"Saved {GOLD_FEATURES_TABLE}")

In [0]:
GOLD_EMBEDDINGS_TABLE = "anime_project.gold.anime_embeddings"

df_with_embeddings_spark \
    .select(
        "user_id",
        "anime_id",
        "anime_title",
        "user_score",
        "synopsis_embedding"
    ) \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(GOLD_EMBEDDINGS_TABLE)

print(f"Saved {GOLD_EMBEDDINGS_TABLE}")

In [0]:
%sql
ALTER TABLE anime_project.gold.anime_features

In [0]:
features_count = spark.table(GOLD_FEATURES_TABLE).count()
embeddings_count = spark.table(GOLD_EMBEDDINGS_TABLE).count()

print(f"Features table rows: {features_count}")
print(f"Embeddings table rows: {embeddings_count}")

In [0]:
print("\n--- FEATURES SAMPLE ---")
spark.table(GOLD_FEATURES_TABLE).select(
    "anime_id",
    "anime_title",
    "user_score",
    "user_preference_alignment",
    "external_quality_score"
).show(3)

print("\n--- EMBEDDINGS SAMPLE ---")
spark.table(GOLD_EMBEDDINGS_TABLE).select(
    "anime_id",
    "anime_title",
    "user_score",
    "synopsis_embedding"
).show(2, truncate=50)

In [0]:
print(f"\nGold Tables Created:")
print(f"  1. {GOLD_FEATURES_TABLE}")
print(f"  2. {GOLD_EMBEDDINGS_TABLE}")